In [1]:
import torch

/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib

In [4]:
q = torch.distributions.Normal(2, 4)

In [6]:
z = q.rsample()
print(z)

tensor(6.0532)


In [7]:
p = torch.distributions.Normal(0, 1)
q = torch.distributions.Normal(2, 4)

log_pz = p.log_prob(z)
log_qzx = q.log_prob(z)

print('log prob pz: ', log_pz, 'prob:', torch.exp(log_pz))
print('log prob qzx: ', log_qzx, 'prob:', torch.exp(log_qzx))

log prob pz:  tensor(-19.2396) prob: tensor(4.4089e-09)
log prob qzx:  tensor(-2.8186) prob: tensor(0.0597)


In [8]:
kl_divergence = log_qzx - log_pz
kl_divergence

tensor(16.4210)

In [12]:
q = torch.distributions.Normal(0, 1)

log_pz = p.log_prob(z)
log_qzx = q.log_prob(z)

print('log prob pz: ', log_pz, 'prob:', torch.exp(log_pz))
print('log prob qzx: ', log_qzx, 'prob:', torch.exp(log_qzx))
kl_divergence = log_qzx - log_pz
kl_divergence

log prob pz:  tensor(-19.2396) prob: tensor(4.4089e-09)
log prob qzx:  tensor(-19.2396) prob: tensor(4.4089e-09)


tensor(0.)

In [14]:
z1, z2 = q.rsample(), q.rsample()
y = z1 + z2

In [17]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [18]:

from pl_bolts.datamodules import CIFAR10DataModule, ImagenetDataModule

/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
  stdout_func(
/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  stdout_func(
/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `sklearn` which is not installed yet, install it with `pip install sklearn`.
  stdout_func(


In [23]:
import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F
from pl_bolts.models.autoencoders.components import (
    resnet18_decoder,
    resnet18_encoder,
)

class VAE(pl.LightningModule):
    def __init__(self, enc_out_dim=512, latent_dim=256, input_height=32):
        super().__init__()

        self.save_hyperparameters()

        # encoder, decoder
        self.encoder = resnet18_encoder(False, False)
        self.decoder = resnet18_decoder(
            latent_dim=latent_dim, 
            input_height=input_height, 
            first_conv=False, 
            maxpool1=False
        )

        # distribution parameters
        self.fc_mu = nn.Linear(enc_out_dim, latent_dim)
        self.fc_var = nn.Linear(enc_out_dim, latent_dim)

        # for the gaussian likelihood
        self.log_scale = nn.Parameter(torch.Tensor([0.0]))

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def gaussian_likelihood(self, mean, logscale, sample):
        scale = torch.exp(logscale)
        dist = torch.distributions.Normal(mean, scale)
        log_pxz = dist.log_prob(sample)
        return log_pxz.sum(dim=(1, 2, 3))

    def kl_divergence(self, z, mu, std):
        # --------------------------
        # Monte carlo KL divergence
        # --------------------------
        # 1. define the first two probabilities (in this case Normal for both)
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        # 2. get the probabilities from the equation
        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        # kl
        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)
        return kl

    def training_step(self, batch, batch_idx):
        x, _ = batch

        # encode x to get the mu and variance parameters
        x_encoded = self.encoder(x)
        mu, log_var = self.fc_mu(x_encoded), self.fc_var(x_encoded)

        # sample z from q
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()

        # decoded 
        x_hat = vae.decoder(z)

        # reconstruction loss
        recon_loss = self.gaussian_likelihood(x_hat, self.log_scale, x)

        # kl
        kl = self.kl_divergence(z, mu, std)

        # elbo
        elbo = (kl - recon_loss)
        elbo = elbo.mean()

        self.log_dict({
            'elbo': elbo,
            'kl': kl.mean(),
            'recon_loss': recon_loss.mean(), 
            'reconstruction': recon_loss.mean(),
            'kl': kl.mean(),
        })

        return elbo

In [24]:
from pl_bolts.datamodules import CIFAR10DataModule

datamodule = CIFAR10DataModule('.')

In [28]:
pl.seed_everything(1234)

vae = VAE()
trainer = pl.Trainer(gpus=1, max_epochs=30, progress_bar_refresh_rate=10, accelerator='cpu')
trainer.fit(vae, datamodule)

Global seed set to 1234
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170498071/170498071 [03:57<00:00, 717594.76it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


Missing logger folder: /Users/Egor_Osinkin/projects/vae4fashion/lightning_logs
/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:88: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:107: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(

  | Name    | Type          | Params
------------------------------------------
0 | encoder | ResNetEncoder | 11.2 M
1 | decoder | ResNetDecoder | 8.6 M 
2 | fc_mu   | Linear        | 131 K 
3 | fc_var  | Linear        | 131 K 
------------------------------------------
20.1 M    Trainable params
0         Non-trainable param

Epoch 0:   2%|██▎                                                                                               | 30/1250 [02:11<1:28:53,  4.37s/it, loss=3.08e+03, v_num=0]

/Users/Egor_Osinkin/Library/Caches/pypoetry/virtualenvs/vae4fashion-tiTJ48CW-py3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
